In [1]:
import os
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from utils.dataloader import DataLoader
from utils.vis import MatplotlibVisualizer
from utils.transforms import HairRemoval, Composer
from descriptors.texture import LBPDescriptor, GLCMDescriptor
from utils.segmentation import ThresholdingSegmentation
from descriptors.bow import MultifeatureBoW
from descriptors.color import ColorDescriptor, ColorLayoutDescriptor, ColorCooccurrenceMatrixDescriptor

In [2]:
## Classes
CLASSES = ['nevus', 'others']

## Work folfer
work_folder = os.getcwd()
data_folder = os.path.join(work_folder, '..', 'Data/Challenge1')

## Visualizer
matplotlib_visualizer = MatplotlibVisualizer()

In [10]:
color_descriptor = ColorDescriptor(bins=(8, 12, 3))
color_cooccurrence_matrix_descriptor = ColorCooccurrenceMatrixDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8)
glcm_descriptor = GLCMDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8, visualize=False, grid_x=1, grid_y=1)
lbp_descriptor = LBPDescriptor(radius=2, n_points=8, grid_x=1, grid_y=1, visualize=False)

descriptors = [lbp_descriptor]
classifier = xgb.XGBClassifier(objective='binary:logistic', n_estimators=1000, learning_rate=0.1, n_jobs=-1)

# Initialize the MultifeatureBoW model
multifeature_bow = MultifeatureBoW(vocab_size=500, descriptors=descriptors)

In [11]:
train_dataloader = DataLoader(data_folder, 'train', 
                        shuffle=True, 
                        ignore_folders=['black_background', '.DS_Store'], 
                        max_samples=None, 
                        balance=False,
                        transforms=None,
                        classes=CLASSES,
                        mask=False)

# Build the codebook using the training data
multifeature_bow.build_codebook(train_dataloader)


# Train the classifier using BoW histograms
multifeature_bow.fit_classifier(train_dataloader)

Extracting Features for LBPDescriptor: 100%|██████████| 15195/15195 [13:56<00:00, 18.16it/s]


Fitting classifier...


Transforming Images: 100%|██████████| 15195/15195 [14:05<00:00, 17.96it/s]


In [ ]:
test_dataloader = DataLoader(data_folder, 'val', 
                        shuffle=True, 
                        ignore_folders=['black_background', '.DS_Store'], 
                        max_samples=None, 
                        balance=False,
                        transforms=None,
                        classes=CLASSES, 
                        mask=False)

predictions, labels = multifeature_bow.evaluate(test_dataloader, class_names=CLASSES)

Evaluating...
Predicting...


Transforming Images: 100%|██████████| 3796/3796 [03:30<00:00, 18.05it/s]


In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(labels, predictions))
print(confusion_matrix(labels, predictions))
print(classification_report(labels, predictions, target_names=CLASSES))

0.6812434141201265
[[1307  624]
 [ 586 1279]]
              precision    recall  f1-score   support

       nevus       0.69      0.68      0.68      1931
      others       0.67      0.69      0.68      1865

    accuracy                           0.68      3796
   macro avg       0.68      0.68      0.68      3796
weighted avg       0.68      0.68      0.68      3796

